In [3]:
import numpy as np
import psycopg2
import matplotlib.pyplot as plt

# Connect to DB containing the exported tables using import-osm
# Based on martinelli's work
connection = psycopg2.connect(
    database="noise",
    user="noise",
    password="noise",
    host="localhost",
    port=63559 # Change to the port postgres is accessible from
)
cursor = connection.cursor()

## Query
This query gets all geometries within x meters of certain long, lat of a certain type

In [4]:
q = """
SELECT
    id,
    ST_Distance(ST_Transform(ST_SetSrid(ST_Point({0}, {1}), 4326), 3857), geometry) * cosd({1}) dist
    {5}
FROM {2}
WHERE
    ST_DWithin(ST_Transform(ST_SetSrid(ST_Point({0}, {1}), 4326), 3857), geometry, {3} / cosd({1}))
ORDER BY dist
LIMIT {4}
"""

## Config
* tables: the table names containing the geometries. Mapping to the noise produced by each geometry
* limits: How many rows to retreive from each table for measuring noise

In [5]:
tables = {
    "osm_light_railway": 70,
    "osm_secondary_road": 85,
    "osm_trunk": 95,
    "osm_motorway": 100,
    "osm_tertiary_road": 88,
    "osm_primary_road": 90,
    "osm_railway": 90,
    "osm_retail_zone": 80,
    "osm_sport": 70,
    "osm_sport_point": 70,
    "osm_leisure": 60,
    "osm_party": 80,
    "osm_shop": 60,
    "osm_industrial_zone": 80,
    "osm_food": 65,
    "osm_school": 70,
    "osm_school_point": 70
}
limits = {
    "osm_light_railway": 1,
    "osm_secondary_road": 2,
    "osm_trunk": 2,
    "osm_motorway": 2,
    "osm_tertiary_road": 2,
    "osm_primary_road": 2,
    "osm_railway": 1,
    "osm_retail_zone": 2,
    "osm_sport": 2,
    "osm_sport_point": 2,
    "osm_leisure": 2,
    "osm_party": 2,
    "osm_shop": 2,
    "osm_industrial_zone": 2,
    "osm_food": 2,
    "osm_school": 2,
    "osm_school_point": 2,
}
speed_limits = {
    "osm_trunk": [100, 80],
    "osm_motorway": [100, 80],
    "osm_secondary_road": [50 ,30],
    "osm_tertiary_road": [50, 30],
    "osm_primary_road": [50, 30],
}

## Endpoint

A function that calculates noise given long, lat

In [6]:
def noise(long, lat, tables, max_dist=1200):
    """Noise report for certain long, lat
    
    Args:
        long: Longitude
        lat: Latitude
        max_dist: Maximum distance to look for geometries for (in meters)
        
    Returns:
        dict containing the report of the structure (noise in dB and distances in meters)
        {'osm_secondary_road': 166,
         'osm_tertiary_road': 355,
         'osm_primary_road': 234,
         'osm_railway': 184,
         'osm_retail_zone': 272,
         'osm_party': 234,
         'osm_shop': 163,
         'osm_industrial_zone': 272,
         'osm_food': 271,
         'noise': 49.9986629248497}
    """
    dist  = [] # Distances
    power = [] # Noise Power
    info  = {} # Nearest of each type
    
    # get nearest x geometries of each type
    for table in tables:
        extention = ""
        if table in speed_limits:
            extention = ", maxspeed"
        cursor.execute(q.format(long, lat, table, max_dist, limits[table], extention))
        record = cursor.fetchall()
        for r in record:
            dist.append(round(r[1]))
            power.append(tables[table])
            if table in speed_limits and r[2] and r[2] <= speed_limits[table][0]:
                power[-1] -= 5
            if table in speed_limits and r[2] and r[2] <= speed_limits[table][1]:
                power[-1] -= 5
            info[table] = info.get(table, round(r[1]))

    # Noise calculation formula (simple noise propagations)
    dist = np.array(dist).clip(5)
    power = np.array(power)
    info["noise"] = np.log10((10**(power / 10 - 2 * abs(np.log10(dist)) - dist / 3000)).sum()) * 10
    return info

In [7]:
%%time
noise(8.451002, 50.092927, tables)

CPU times: user 2.16 ms, sys: 2.78 ms, total: 4.94 ms
Wall time: 48.7 ms


{'osm_secondary_road': 410,
 'osm_tertiary_road': 369,
 'osm_primary_road': 233,
 'osm_railway': 1066,
 'osm_retail_zone': 170,
 'osm_sport': 121,
 'osm_sport_point': 888,
 'osm_leisure': 1061,
 'osm_party': 733,
 'osm_shop': 259,
 'osm_industrial_zone': 339,
 'osm_food': 251,
 'osm_school': 158,
 'osm_school_point': 747,
 'noise': 42.5120735798405}